In [1]:
from importlib.resources import files

import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

from aml import models

In [2]:
model_path = str(files(models).joinpath("paraphrase-MiniLM-L3-v2"))

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:\Users\Colin\Documents\akas\ml\aml\src\aml\models\paraphrase-MiniLM-L3-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
dataset = load_dataset("sst2", split="train")

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(400))
small_eval_dataset = tokenized_datasets.shuffle(seed=43).select(range(100))

In [ ]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=-1)
    return {"acc": np.mean(preds == labels)}

training_args = TrainingArguments(
    output_dir="test_trainer", 
    save_strategy="no", 
    eval_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [7]:
trainer.train()

c:\Users\Colin\Documents\akas\ml\aml\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Acc
1,No log,0.633648,0.590000
2,No log,0.479594,0.810000
3,No log,0.440232,0.820000


TrainOutput(global_step=150, training_loss=0.5249308268229167, metrics={'train_runtime': 50.156, 'train_samples_per_second': 23.925, 'train_steps_per_second': 2.991, 'total_flos': 5043704832000.0, 'train_loss': 0.5249308268229167, 'epoch': 3.0})

In [8]:
# Task 1
# Write a method that takes a text and makes a prediction

In [9]:
# Look into the parameters id2label and label2id for the from_pretrained method

In [10]:
# Try out a different dataset

In [11]:
# Run the code on colab, on a gpu

In [12]:
# Try out a larger model like "google-bert/bert-base-uncased"